# Data Transformation and Preparation

In [ ]:
import pandas as pd

# read the data
spotify = pd.read_csv('data/Hot 100 Audio Features.csv')
billboard = pd.read_csv('data/Hot Stuff.csv')

# join on song title and performer
spotify = spotify.drop_duplicates(subset=['Song', 'Performer']) # drop duplicates in spotify
df = pd.merge(spotify, billboard, on=['Song', 'Performer'], how='right', validate='one_to_many')

29503 327895
327895
